# Doc2Vec을 이용한 문서 유사도 측정
- word2vec의 개념을 단어가 아니라 **문구**나 **문장**, **문서 전체**로 확장 가능.
- 기존 단어들에 근거해서 단어를 예측함으로써 단어 벡터들을 학습한다는 착안(word2vec) -> 문장, 문단, 문서 벡터의 학습으로 확장 가능

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns 
import re
plt.style.use('seaborn-whitegrid')
plt.rcParams['font.family'] = 'Malgun Gothic' 
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = 10,5
plt.rcParams['axes.unicode_minus'] = False

In [3]:
import multiprocessing
num_cores = multiprocessing.cpu_count()

from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec, Doc2VecVocab, Doc2VecTrainables
from gensim.utils import simple_preprocess

df =  pd.read_csv('../2030표심 및 정치성향 분석/data/전처리데이터/data2030_전처리_좋아요가중치_텍스트전처리.csv')
corpus = df['Comment_textPreprocessing'].dropna( axis=0).to_list()

In [5]:
# Null값 처리
len(corpus)
corpus[:5]

['내리막이 있으면 다시 오르막이 있는 법, 예방주사라 생각하고 힘들 냅시다.',
 '2030 마음을 얻으려면 조언을 2030을 모셔다가 들어보는 성의를 보여야 합니다. 2,030도 아닌 사람들이 추측해서 2030의 마음을 이야기하는 것 자체가 문제의식이 없는 것입니다. 아마추어라도 2030사람들을 몇 번이라도 불러서 보여주며 실질적인 것을 들어보는 것이 좋다고 생각해요.',
 '전 군대 전역 전까지는 보수를 지지했지만, 대학에서 양극화에 관심을 가지면서 진보로 돌아섰죠! 20대의 경우 성격이 너무나 다르기에 초중반을 나눠서 공략해야 한다고 생각해요! 가령 20대 초반이 등록금 같은 게 문제라면 중반은 취업 후반은 자산 형성처럼 2,030대 고민 혼재 세세하게 고민하는 흔적들을 보여주는 게 중요하다고 생각합니다... 정당 색채가 엷을 때는 사소한 이유로 지지 정당이 변하는 경우가 많으니….',
 '몸 안에서 무엇인가 무너져 내린 기분.. 그래도 힘내고 가야죠. 민주당은 뼈아픈 진단 새겨들어야 합니다. 황희 두 임 자주 함께해서 2,030대 남성들의 마음을 얻을 방안도 모색해야 합니다. 그리고 전 국민 재난지원금 보편 지급!! 언론개혁!!',
 '네 죄송합니다. 삭제하겠습니다']

# 학습시킬 문장 토큰화하기

In [ ]:
test_sentencs = ['21분 생활권 도시 서울, 세계 디지털 경제수도 서울 구현',
'21분안에 주거, 직장, 쇼핑., 여가, 건강, 의료, 교육, 보육이 해결되는 도시',
'강남 강북 균형발전 추진',
'대전환펀드 1조원 구성',
'블록체인 기술 기반 KS코인 운용. Al,5G 등 철단기술이 융합된 경제도시 조성',

'집 격정 없는 서울, 클러스터 중심 일자리창출과 서울형 창업생태계 조성',
'평당 천만원 반값아파트 공공주택 30만호 공급',
'청년 등 전월세 보증금 무이자 지원, 최저주거기준 주택 개선자금 지원',
'소상공인 임대료 30% 감면, 임대업자 15% 시비 지원',
'특수고용직, 플랫폼노동자, 프리랜서 등 고용보험료 지원',

'기후와 환경, 교통 대전환',
'21분 녹색길 조성',
'지하철 1,2,4호선 경의중앙선 지상구간 지하화',
'경부고속도로 양재-한남구간, 동서부간선도로 지하화',
'2030년부터 내연차 신규등록 금지',

'대상별 맞춤형 돌봄과 교육으로 건강하고 행복한 공동체 서울',
'유치원 전면 무상급식 추진-어린이집 교사대비 아동비율 개선',
'장애인, 어르신을 위한 돌봄SOS센터 인력확충',
'원스톱 헬스케어 센터 중심 의료서비스 확대',

'재난과 위기에 강한 서울, 세계 문화의 새로운 중심 서울',
'문화 패스 카드 도입과 확대, 예술인 고용산재보험 가입지원',
'시민주도형 재난안전 협의체 구성등으로 안전관리체계 개편',
'동물병원 진료항목 표준화, 진료비 공시제 도입']

In [43]:
corpus

def read_corpus(fname, token_only=False):
    for idx, line in enumerate(fname):
        tokens = gensim.utils.simple_preprocess(line)
        if token_only:
            yield tokens
        else:
            yield gensim.models.doc2vec.TaggedDocument( tokens, [idx])

training_data = list(read_corpus(corpus))
testing_data = list(read_corpus(test_sentencs,token_only=True))

In [91]:
training_data

[TaggedDocument(words=['내리막이', '있으면', '다시', '오르막이', '있는', '예방주사라', '생각하고', '힘들', '냅시다'], tags=[0]),
 TaggedDocument(words=['마음을', '얻으려면', '조언을', '모셔다가', '들어보는', '성의를', '보여야', '합니다', '아닌', '사람들이', '추측해서', '마음을', '이야기하는', '자체가', '문제의식이', '없는', '것입니다', '아마추어라도', '사람들을', '번이라도', '불러서', '보여주며', '실질적인', '것을', '들어보는', '것이', '좋다고', '생각해요'], tags=[1]),
 TaggedDocument(words=['군대', '전역', '전까지는', '보수를', '지지했지만', '대학에서', '양극화에', '관심을', '가지면서', '진보로', '돌아섰죠', '대의', '경우', '성격이', '너무나', '다르기에', '초중반을', '나눠서', '공략해야', '한다고', '생각해요', '가령', '초반이', '등록금', '같은', '문제라면', '중반은', '취업', '후반은', '자산', '형성처럼', '고민', '혼재', '세세하게', '고민하는', '흔적들을', '보여주는', '중요하다고', '생각합니다', '정당', '색채가', '엷을', '때는', '사소한', '이유로', '지지', '정당이', '변하는', '경우가', '많으니'], tags=[2]),
 TaggedDocument(words=['안에서', '무엇인가', '무너져', '내린', '기분', '그래도', '힘내고', '가야죠', '민주당은', '뼈아픈', '진단', '새겨들어야', '합니다', '황희', '자주', '함께해서', '남성들의', '마음을', '얻을', '방안도', '모색해야', '합니다', '그리고', '국민', '재난지원금', '보편', '지급', '언론개혁'], tags=[3]),
 TaggedDocument(words=['죄송합니다'

# 모델 훈련시키기

In [24]:
# 모델 생성
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=10)
model

In [27]:
# 사전 생성
model.build_vocab(training_data)

In [28]:
# 패널티 확인
print(f"Word 'penalty' appeared {model.wv.get_vecattr('penalty', 'count')} times in the training corpus.")

KeyError: "Key 'penalty' not present"

In [30]:
# 훈련
model.train(training_data, total_examples=model.corpus_count, epochs=model.epochs)

In [31]:
# 새로운 문서에 대한 벡터 추론
vector = model.infer_vector(['부동산 집값을 잡는 사람 어디 없나'])
print(vector)

[ 0.00085917  0.00229793  0.00292367  0.00470927  0.00084832 -0.00257527
  0.00327176 -0.00283191 -0.00160654  0.00086754 -0.00018763  0.00166592
 -0.00344948 -0.00141881  0.00397145 -0.00336343 -0.00468203 -0.0023099
  0.00195858  0.00030518 -0.00427272  0.00481156  0.00161364 -0.00463759
  0.0005946   0.00021767 -0.00424622  0.0013695  -0.00414187  0.00010172
 -0.00120843  0.00377046  0.00430842  0.00192175 -0.00464699  0.00189064
 -0.00260742  0.00342821  0.0045185   0.0027479  -0.00350343 -0.00113609
  0.00289138  0.00080087 -0.00328882 -0.00026577 -0.0047236  -0.00279448
 -0.00155899 -0.00409005  0.00084722 -0.00022525 -0.00031188 -0.00098193
 -0.00121044 -0.00083029  0.00239051  0.00325573  0.00229678  0.00120379
 -0.00394575  0.00438    -0.0002592  -0.00199042  0.00462361 -0.00287693
 -0.00036828 -0.00014724  0.00188584 -0.0033101   0.00062255  0.00214339
 -0.00327814 -0.00417578 -0.00096717 -0.00214104  0.00116852 -0.00249785
  0.00125399  0.00184956  0.0035612   0.00288964  0.

In [132]:
# 테스트해보기
from numpy import random

# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(testing_data) - 1)
inferred_vector = model.infer_vector(testing_data[doc_id])
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
# print(sims)
# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}\»\n'.format(doc_id, ' '.join(testing_data[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    # print(f'')
    print(f'{label}, {sims[index]}:«%s»\n' %(" ".join(training_data[sims[index][0]].words)) )
    # print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(testing_data[sims[index][0]].words)))

Test Document (3): «대전환펀드 조원 구성»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d100,n5,w5,mc2,s0.001,t3):

MOST, (17385, 0.9689297676086426):«박영선에게 부릅니다 소녀시대 어리다고 놀리지 말아야 그러게 무시 하래 ㅋㅋㅋㅋㅋ»

MEDIAN, (21683, 0.7253535389900208):«아침잠이 많아서 질질 시간 끌면서 자다가 어쩔 없이 마지막 순간에 일어나는 나의 모닝 도전»

LEAST, (50136, -0.6933519244194031):«미래를 박살 내고 있는데 진보 보수가 중요하냐고 크크크»



In [126]:
pd.read_csv('../2030표심 및 정치성향 분석/data/KR-Word데이터/keyword_keysentence.csv', index_col=0)

,keyword,keysentence
0,언론,협치하다간 정권 뺏긴다. 검찰 개혁 언론개혁 확실히 해야 한다. 재난지원금은 보편 ...
1,20대,"민생 부동산 문제도 너무나 중요하지만, 언론과 검찰 개혁은 민주당의 존립이 달린 문..."
2,초선,요대로 180석 안일하게 버티고 있으면 내년 대선? 후! 저부터 빨간 당으로 갑니다...
3,개혁,야당과 왜 협치해? 그냥 밀고 나가!! 부동산 적폐. 검찰 개혁. 언론 개혁
4,선거,"검찰, 언론개혁 안 해서 진 줄 모르고 멍청한 소리 한 거죠 촛불 시민을 적의로 돌..."
...,...,...
295,가야,초선 의원들 모임 가시면 1. 다선들 자제시켜라 2. 초선들이 직접 나서라 3. 네...
296,마라.,"민동기 기자의 주장이나 분석은 일면 동의하기도 하나... 언론, 포털에 대한 확실한..."
297,실망,저 중고딩 아이 둘 키우는 40대 중반 주부지만 이삼십대 여자들 요즘 혜택이 더 많...
298,1주택,그냥 돈을 주세요. 2030 돈이 없어요. 코로나로 인건비부터 줄였습니다. 2030...
